<img src="images\Logo_UCLL_ENG_RGB.png" style="background-color:white;" />

# Data Analytics & Machine Learning

Lecturers: Aimée Lynn Backiel, Kenric Borgelioen, and Daan Nijs

Academic year 2024-2025

## Lab 6: Machine learning, part 2

### Lecture outline

1. Recap of previous weeks
2. Overfitting and underfitting
3. Automating machine learning pipelines with sci-kit learn
4. Model evaluation 
5. parameters and hyperparameters

### Recap of last lecture(s)

#### Lab 1

1. We ensured we had a valid Python installation.
2. We learnt what a virtual environment is:
   * Isolated Python executable and packages.
   * We created a virtual environment.
3. Absolute path vs relative path recap.
4. Recap of data structures in Python

#### Lab 2
1. Installed Pandas
2. Learnt how to read data
3. Learnt how to calculate mean, mode, median etc.
4. Basic exploration of the 4 variables

#### Lab 3
1. Wrapped up computing summary statistics (mean, median, mode, ...)
2. Learnt how to deal with outliers 
3. Focused on exploration of dat

#### Lab 4
1. Univariate data visualization using Matplotlib
   1. Figures and axes
   2. Histograms
   3. Box plots
   4. Bar charts
2. Multivariate data visualization using Seaborn
   1. Scatter plots
   2. Small multiples
   3. Color coding

#### Lab 5
2. Intro to machine learning using scikit-learn
   1. Preprocessing
      1. One Hot encoding
      2. Scaling
      3. Outliers
   2. Classification and regression

### The case

Ada Turing Travelogue, or as everyone calls her, Ada just started working part time at her parents travel agency. She has a keen understanding and interest of everything related to applied computer science ranging from server & system management to full stack software development. Through database foundations she already understands how to query data and programming 1 and 2 covered the essentials about the Python programming language. Recently she has just decided to start learning about data analytics & machine learning as well.

She uses her skills to connect to the travel agency's database where she finds many, normalized, tables. Ada recalls what she learnt in database foundations and performs all the correct joins. Afterwards she saves the data in the `data/` folder.


She finds the following dataset:

| Column Name          | Description                                                                                       |
| -------------------- | ------------------------------------------------------------------------------------------------- |
| SalesID              | Unique identifier for each sale.                                                                  |
| Age                  | Age of the traveler.                                                                              |
| Country              | Country of origin of the traveler.                                                                |
| Membership_Status    | Membership level of the traveler in the booking system; could be 'standard', 'silver', or 'gold'. |
| Previous_Purchases   | Number of previous bookings made by the traveler.                                                 |
| Destination          | Travel destination chosen by the traveler.                                                        |
| Stay_length          | Duration of stay at the destination.                                                              |
| Guests               | Number of guests traveling (including the primary traveler).                                             |
| Travel_month         | Month in which the travel is scheduled.                                                           |
| Months_before_travel | Number of months prior to travel that the booking was made.                                       |
| Earlybird_discount   | Boolean flag indicating whether the traveler received an early bird discount.                     |
| Package_Type         | Type of travel package chosen by the traveler.                                                    |
| Cost                 | Calculated cost of the travel package.                                                            |
| Margin | The cost (for the traveler) - what the travel agency pays. |
 | Additional_Services_Cost| The amount of additional services (towels, car rentals, room service, ...) that was bought during the trip. |


#### Our challenge

Before getting into harder use cases we will start off by predicting the cost of a given stay. Right now Ada's parents do this manually automating this task would already be a big help to their business.



<center>
<img src="https://www.datascience-pm.com/wp-content/uploads/2021/02/CRISP-DM.png" style="background-color:white;width:50%">
</center>

It also helps to situate our progress within CRISP-DM. We have done the first three steps, as from this lecture we will progress to modeling. As mentioned in the lecture, this is an iterative procedure, as we are doing modeling we need to circle back to both data preparation.

### Machine learning with sci-kit learn


<center>
<img src="https://scikit-learn.org/stable/_images/grid_search_workflow.png" style="background-color:white;width:50%">
</center>

##### ❓ What have we done so far of the image below? What stages have we completed?


We have done train test splitting and we have built a few models on the training set. 

#### Recap overfitting and underfitting

Overfitting is when the model doesn't learn general patterns from the data but rather focuses on doing really well on the training data. 

❗ Typically it means that the performance on the test set will be a lot worse than that on the training set.

Underfitting is when the model learns a pattern that doesn't significantly capture the details of the training set.

❗ Typically it means that the performance on the test set will be similarly (bad) on as that on the training set.

<center>
<img src="https://www.mathworks.com/discovery/overfitting/_jcr_content/mainParsys/image.adapt.full.medium.svg/1686825007300.svg" style="background-color:white;width:50%">
</center>

Let's try this out continuing where we left of last session.

❗ While we do this section we will also make a number of methodological mistakes. We do this for two main reasons:
* To show how easily "mistakes" can be made in machine learning.
*  To introduce Pipelines later on which have cleaner syntax and protect you from a lot of things you could do incorrectly.

In [ ]:
import pandas as pd # by convention
pd.options.display.float_format = '{:.2f}'.format
from sklearn.model_selection import train_test_split
import plotly.express as px
import numpy  as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler


In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor

In [ ]:
def plot_results(y_true: pd.Series, predictions: np.ndarray, title: str) -> None:
    
    fig = px.scatter(x=predictions, y=y_true, labels={"x": "predicted", "y": "actual"}, title=title)
    fig.add_shape(type="line",
                x0=-1000, 
                y0=-1000, 
                x1=7000, 
                y1=7000)
    
    fig.show()

In [ ]:
travel_dataset = pd.read_csv("data/lab_6_dataset.csv")
X = travel_dataset.drop(columns="cost") # by convention the variables are called (capital) X
y = travel_dataset["cost"] # By convention what you want to predict is called (small letter) y

In [ ]:
# Splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=42)

In [ ]:
cat_columns = ["package_Type", "destination", "country"]
numeric_columns = ["guests", "age", "stay_length"]


In [ ]:
def preprocess_data(data: np.ndarray, categorical_columns: list[str], numeric_columns: list[str]) -> np.ndarray:

    """Preprocess the test data by applying a scaling operation on the numeric columns and a one hot encoding on the categorical columns.

    data, np.ndarray: the data you wish to transform.
    categorical_columns, list[str]: the categorical columns that need to be one-hot encoded.
    numeric_columns, list[str]: the numeric columns that need to be scaled.

    Returns:
        np.ndarray: The
    """

    # One hot encoding on the categorical columns
    ohe = OneHotEncoder(sparse_output=False)
    ohe.fit(data[cat_columns])
    cat_cols_train  = ohe.transform(data[categorical_columns])

    # Standard scale the numeric columns
    scaler = StandardScaler()
    num_cols_train = scaler.fit_transform(data[numeric_columns])

    return np.hstack((cat_cols_train, num_cols_train))

In [ ]:
X_train_preprocessed = preprocess_data(X_train, cat_columns, numeric_columns)
X_test_preprocessed = preprocess_data(X_test, cat_columns, numeric_columns)

In [ ]:
lin_reg = LinearRegression()
lin_reg.fit(X_train_preprocessed, y_train)
predictions_lin_reg = lin_reg.predict(X_train_preprocessed)
predictions_test_lin_reg= lin_reg.predict(X_test_preprocessed)

In [ ]:
plot_results(y_train, predictions_lin_reg, title="predicted vs actual for linear regression on the training set")

In [ ]:
plot_results(y_test, predictions_test_lin_reg, title="predicted versus actual on the test set for linear regression")

In [ ]:
decision_tree = DecisionTreeRegressor()
decision_tree.fit(X_train_preprocessed, y_train)
predictions_dt = decision_tree.predict(X_train_preprocessed)
predictions_test_dt = decision_tree.predict(X_test_preprocessed)

In [ ]:
plot_results(y_train, predictions_dt, title="predicted versus actual on the training set for a decision tree")

In [ ]:
plot_results(y_test, predictions_test_dt, title="predicted versus actual on the test set for a decision tree")

##### ❓ Which of the two models is overfitting? Can you describe how and why?

##### ❓ Which of the two models is underfitting? Can you describe how and why?

#### Using pipelines to rectify our mistake

In machine learning, the preprocessing step is critical to prepare the data for modeling. However, there's a subtle but crucial methodological error in the function shown. The issue lies in the lines where `OneHotEncoder` and `StandardScaler` are fitted:

```python
def preprocess_data(data: np.ndarray, categorical_columns: list[str], numeric_columns: list[str]) -> np.ndarray:

    ohe = OneHotEncoder(sparse_output=False) 
    ohe.fit(data[cat_columns]) # The error occurs here!
    cat_cols_train  = ohe.transform(data[categorical_columns])


    scaler = StandardScaler()
    num_cols_train = scaler.fit_transform(data[numeric_columns]) # And here!

    return np.hstack((cat_cols_train, num_cols_train))  
```

The key mistake in the preprocessing function provided lies in fitting the OneHotEncoder and StandardScaler to the data inside the function. This could lead to a situation where, when preprocessing the test set, the encoders and scalers are fitted again separately with the test set statistics, which is incorrect. The preprocessing steps that "learn" from the data, such as encoding categorical variables and scaling numerical variables, should be based solely on the training data to avoid introducing bias from the test set into the model.

The proper methodology is to fit the OneHotEncoder and StandardScaler on the training data only. This way, they "learn" the categories of the categorical variables and the distribution (mean and standard deviation) of the numerical variables from the training set. Once fitted, these preprocessors should then be applied to the test data, ensuring that the transformation applied is consistent and does not give the model any information about the test set.

One potential way to solve it is as follows:

```python
def fit_preprocessors(train_data: np.ndarray, categorical_columns: list[str], numeric_columns: list[str]) -> (OneHotEncoder, StandardScaler):
    # Fit the OneHotEncoder and StandardScaler to the training data
    ohe = OneHotEncoder(sparse_output=False)
    scaler = StandardScaler()
    ohe.fit(train_data[categorical_columns])
    scaler.fit(train_data[numeric_columns])
    
    # Return the fitted preprocessors
    return ohe, scaler

def transform_data(data: np.ndarray, categorical_columns: list[str], numeric_columns: list[str], ohe: OneHotEncoder, scaler: StandardScaler) -> np.ndarray:
    # Transform data using the already fitted preprocessors
    cat_cols = ohe.transform(data[categorical_columns])
    num_cols = scaler.transform(data[numeric_columns])
    
    # Return the transformed data
    return np.hstack((cat_cols, num_cols))

```

In scikit-learn, the Pipeline and ColumnTransformer classes offer an idiomatic and streamlined way to chain multiple preprocessing steps and a model into a single workflow. The `Pipeline` class allows you to assemble sequences of transformations and a final model, which simplifies your code and helps prevent common mistakes, such as fitting preprocessing steps to the test data. The `ColumnTransformer` is particularly useful for applying different preprocessing to different columns, such as one-hot encoding for categorical variables and scaling for numerical variables. By combining these tools, you not only reduce the need for manual 'glue' code but also safeguard against the leakage of information from the test set into the training process. We highly recommend you always use this in the scope of this course.



The syntax is quite simple:

*  `make_column_transformer` expects multiple tuples. The `transformer` (preprocessing step) is in the first position of the tuple and the columns you apply the transformer (type: `list[str]`) is in the second position.  
*  `make_pipeline` similarly expects all objects in sequence, so typically you add your preprocessing first followed by the model you want to apply. 

In [ ]:
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline

In [ ]:
preprocessing = make_column_transformer(
    (StandardScaler(), numeric_columns),
    (OneHotEncoder(sparse_output=False), cat_columns),
    remainder="drop"
)

In [ ]:
preprocessing.fit_transform(X_train)

In [ ]:
lin_reg_pipe = make_pipeline(preprocessing, LinearRegression())

In [ ]:
lin_reg_pipe.fit(X_train, y_train)
predictions_lin_reg_train = lin_reg_pipe.predict(X_train)
predictions_lin_reg_test = lin_reg_pipe.predict(X_test)

That's all!

Convince yourself for a second that is a lot simpler than what we were doing previously, we simply use `make_column_transformer` to indicate what preprocessing we want to apply to which column. Afterwards we put our preprocessing in a pipeline with the model we want to use.

❗ Once you call `model.fit(X_train, y_train)` it both fits the preprocessing and the model in one go. 

Pipelines make it easy to make many different models in one go. 

In [ ]:
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor

In [ ]:
decision_tree_pipe = make_pipeline(preprocessing, DecisionTreeRegressor())
rf_pipe = make_pipeline(preprocessing, RandomForestRegressor())
xgb_pipe = make_pipeline(preprocessing, HistGradientBoostingRegressor())

You can even do the following if you want:

```python

model_name_pair = [("random_forest", RandomForestRegressor()), ("gradient boosting", HistGradientBoostingRegressor()), ("decision tree", DecisionTreeRegressor())]
results = []
for pair in model_name_pair:
    name, model = pair
    pipe = make_pipeline(preprocessing, model)
    pipe.fit(X_train, y_train)
    predictions_train = pipe.predict(X_train)
    predictions_test = pipe.predict(X_test)
    result.append([(name, predictions_train, predictions_test)])
```

The code above would train 3 models in a single for loop. Afterwards you can use a single function to evaluate the results you have obtained from all three models.

#### Model evaluation

Up until now we've only *qualitatively* judged the quality of our models by looking at our predicted versus actual plot. We're interested in having a single number that summarizes the the performance of our model. The reason is that investigating graphs doesn't scale well.

##### Attempt 0: taking the mean of the error

Our first intuition might be to take the difference of the predictions and the actual values. This is called the **error** or the **residual**. After we have this value we may be tempted to take the mean. 

In [ ]:
error = predictions_lin_reg_test - y_test
error

In [ ]:
np.mean(error)

In [ ]:
px.histogram(error, title="distribution of the errors of linear regression on the test set.")

##### ❓ Why is the mean of residuals misleading?


##### ❓ How can we better quantify errors?

##### Attempt 1: taking the mean of the absolute error (MAE)

In [ ]:
np.mean(np.abs(error))

In [ ]:
px.histogram(np.abs(error), title="distribution of the absolute errors of linear regression on the test set.")

This approach is more informative as it provides a clearer picture of how much error is present on average in our predictions.

By considering the MAE, we obtain a useful summary statistic for model performance that is easy to understand and directly interpretable in terms of the problem at hand.

In practice squaring the errors is more common.

##### Attempt 2: Taking the mean of the squared errors (MSE)

Squaring the errors, as shown in the equation: $(-2)² = 4 = (2)²$, ensures that all error values are positive. This method has two main benefits:

This method has two main advantages:

1. Large errors are amplified more than smaller ones, which can be particularly important in cases where larger deviations are less tolerable.
2. MSE is a differentiable function, which makes it mathematically convenient for optimization algorithms used in model training. There are other statistical properties that the MSE can leverage.

Note: point 2 is only for your information. You don't need to know this at all. 


The MSE can be calculated as follows:

In [ ]:
np.mean(np.square(error))

Since MSE can result in large numbers that are difficult to interpret, we often take the square root to obtain the Root Mean Squared Error (RMSE), which has the same units as the original values:

*Note: the root is $\sqrt(x)$.*

In [ ]:
rmse = np.sqrt(np.mean(np.square(error)))

In [ ]:
px.histogram(np.square(error), title="distribution of the square errors of linear regression on the test set.")

##### ❓ What are downsides of the MSE? There are two, but one is harder to come up with.

##### Summary

Our advice:

* Compute both the MAE and the RMSE. 
* The one you should focus on the most is typically the RMSE. 
* If there are error outliers then the difference between the MAE and the RMSE is likely going to be large. In that case it is typically more interesting to look at the MAE. 
 
Typically, RMSE is preferred because it is more sensitive to large errors, which can be critical in applications where such errors are especially problematic, like in autonomous vehicle guidance systems. However, if your model is prone to outliers, or if large errors are less impactful, MAE can be a more relevant metric. Always consider the specific context of your application when choosing your primary evaluation metric.

##### model evaluation using sci-kit learn


sci-kit learn offers all of these functions out of the box. All you need to do is remember their name.

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error

##### ❓ Use the pipeline approach discussed above and the mean_absolute_error and mean_squared_error for the following models: RandomForestRegressor, HistGradientBoostingRegressor, DecisionTreeRegressor and LinearRegression.

##### ❓ Comment on the behavior of the models. Are they overfitting? Underfitting?

#### The first way to improve our model: feature engineering

Recall in the previous session we made a number of observations:

* If you're visiting the same country as you're from the destination seemed to be cheaper
* If you're traveling in approximately the same continent it's also cheaper
* There might be the case between age and month.
* Maybe we should look at age in groups.

In [ ]:
destination_to_country = {
        "New York": "USA",
        "Rome": "Italy",
        "Paris": "France",
        "Tokyo": "Japan",
        "Cairo": "Egypt",
        "Sydney": "Australia",
        "Rio": "Brazil",
        "Cape Town": "South Africa",
    }
country_to_continent = {
        "USA": "America",
        "UK": "EMEA",
        "France": "EMEA",
        "Canada": "America",
        "Australia": "Asia",
        "Germany": "EMEA",
        "Spain": "EMEA",
        "Italy": "EMEA",
    }
destination_to_continent = {
        "New York": "America",
        "Rome": "EMEA",
        "Paris": "EMEA",
        "Tokyo": "Asia",
        "Cairo": "EMEA",
        "Sydney": "Asia",
        "Rio": "America",
        "Cape Town": "Africa",
    }

##### ❓ Use Pandas to make a variables to indicate if they traveled to the same country and then the same continent. 

##### HINT1: Look at [the map method for Pandas series](https://pandas.pydata.org/docs/reference/api/pandas.Series.map.html). 

##### HINT2: Remember, a series is simply a column so , `df[column]` gives you a series.

When trying to add these variables there is some friction. They don't fit into our sci-kit learn `Pipeline` workflow nicely. We could add them to our entire dataset before splitting. Adding variables to the entire dataset is not risk-free. Doing so may lead to the methodological error we spoke about previously **data leakage**. In the scope of this course it's fine to use this approach to *add* variables. We'll briefly show you the more principled way, but you don't need to know this for the exam. It involves creating a custom `Transformer` which we can then compose in our pipeline as usual.

In [ ]:
from sklearn.base import BaseEstimator, TransformerMixin
from dataclasses import dataclass

@dataclass
class CountryMapping(TransformerMixin, BaseEstimator):
    country_to_continent: dict[str, str]
    destination_to_country: dict[str, str]
    destination_to_continent: dict[str, str]

    def fit(self, X, y=None):
        return self
    
    def transform(self, X: pd.DataFrame):
        _X = X.copy() # so we don't change our input data frame
        _X['country_match'] = _X['country'] == _X['destination'].map(self.destination_to_country)
        _X['continent_match'] = _X['country'].map(self.country_to_continent) == _X['destination'].map(self.destination_to_continent)
        return _X

In [ ]:
country_map = CountryMapping(country_to_continent, destination_to_country, destination_to_continent)

In [ ]:
country_map.fit_transform(X_train)

##### Interaction terms

<img src="https://www.jmp.com/en_se/statistics-knowledge-portal/what-is-multiple-regression/mlr-with-interactions/_jcr_content/par/styledcontainer_2069/par/lightbox_3be9/lightboxImage.img.png/1548351208495.png">

Let's consider an alternative approach to the problem above using interaction terms. Interaction terms are created by multiplying two variables together, effectively creating a new variable. This is a mathematical way of capturing the 'AND' condition in our data. For example, after one-hot encoding categorical variables like 'Country' and 'Destination', we can generate interaction terms to explore the combined effect of these two features.

Suppose we have 'New York', 'Rome', 'Tokyo', and 'Cairo' as categories for 'City' and 'USA', 'Italy', 'Japan', and 'Egypt' for 'Country'. If we create interaction terms for 'City' and 'Country', we end up with additional columns such as 'New York x USA', 'Rome x Italy', and so on. Each of these new columns will have a value of 1 only if both contributing variables (e.g., 'City' is 'New York' AND 'Country' is 'USA') are 1; otherwise, the value will be 0. This new variable thus answers the question: "Is the traveler from X city AND going to Y country?"

By including interaction terms, we allow our model to consider the combined influence of two variables, which can be particularly insightful when the effect of one variable on the outcome depends on the level of another variable.

In scikit-learn, interaction effects between variables can be encoded using the PolynomialFeatures transformer. Interaction effects are valuable in a model when the relationship between two features can affect the outcome in a way that is not simply additive.

For example, consider two binary features, A and B. Individually, they might have a certain effect on the target variable Y. However, when both A and B occur together (i.e., A=1 and B=1), their combined effect on Y could be different from the sum of their individual effects. This is where interaction terms come into play.

The PolynomialFeatures transformer can not only generate polynomial features, which are features raised to a power (like $x^{2}$ or $x^{3}$), but also interaction features, which are products of features (like $x_{1} * x_{2}$).

We will start off by generating interactions between all our numeric columns.

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

poly = PolynomialFeatures(interaction_only=True)

numeric_preprocessing = make_pipeline(poly, StandardScaler())

preprocessing_poly = make_column_transformer(
    (numeric_preprocessing, numeric_columns),
    (OneHotEncoder(sparse_output=False), cat_columns),
    remainder="drop"
)

##### Binning

<img src="https://miro.medium.com/max/2000/1*LGTAObYYj2-fdBMFLz30rw.jpeg" style="width:50%">

Binning is a technique that involves segmenting a continuous variable into several intervals, or 'bins'. Similar to the way we categorize data when creating histograms, binning transforms a continuous variable into an ordered categorical variable. One hot encoding these bins allows us to introduce non-linear effects into our linear models, which ordinarily would interpret the data as having a constant slope.

Take temperature as an example: people generally enjoy mild increases in weather warmth, but there's a threshold beyond which higher temperatures become unpleasant. Binning would let us model this non-linear relationship. Instead of treating temperature as a single continuous predictor with a constant effect, we could divide temperatures into ranges (e.g., 0-10°C, 10-20°C, 20-30°C, etc.) and treat each range as a separate category. By one hot encoding these categories, we enable our model to capture the varying effects of different temperature ranges on people's comfort levels. This approach can reveal more complex patterns in how the predictor variable (in this case, temperature) influences the outcome variable (such as people's reported happiness).




In [ ]:
from sklearn.preprocessing import KBinsDiscretizer


preprocessing_bins = make_column_transformer(
    (StandardScaler(), numeric_columns),
    (KBinsDiscretizer(), "age"),
    (OneHotEncoder(sparse_output=False), cat_columns),
    remainder="drop"
)

preprocessing_poly_bins = make_column_transformer(
    (numeric_preprocessing, numeric_columns),
    (KBinsDiscretizer(), "age"),
    (OneHotEncoder(sparse_output=False), cat_columns),
    remainder="drop"
)

#### Finding the best model: cross validation

As we explore various models and preprocessing techniques, we encounter a dilemma: how do we identify the best model without biasing our selection? 

##### ❓ What is the weakness of the train-test split approach? Think about this before continuing.

Using just one test set might lead us to choose a model that excels on that particular subset of data by sheer luck. What if a different shuffle of the data leads to a different 'best' model?

##### ❓ Think about a solution for this before we continue.

To resolve this, let's consider a more robust method. Imagine if we could test each model not on one but multiple randomized slices of our data. This is where cross-validation comes into play.


Here's how it works: We divide our training set into smaller sections, say 20% chunks. We train our model on 80% of the data, then validate it on the remaining 20%. We repeat this process five times, each time with a different 20% held out for validation. This technique, known as k-fold cross-validation (with k being the number of chunks or 'folds' we create), allows each model a fair shot at proving itself across the entirety of our data.

By averaging the performance across these folds, we obtain a more reliable measure of a model's quality. This thorough approach increases our confidence that we're selecting the best model, not by chance, but by consistent performance.

<center><img src="https://scikit-learn.org/stable/_images/grid_search_cross_validation.png" style="background-color:white"></center>

##### ❓ Sanity check: if we do K-fold cross validation, how many models have we trained. Answer for 2-fold, 3-fold, 5-fold and K-fold.

##### ❓ What is the downside of K-fold cross validation?

As usual, sci-kit learn makes it easy to do cross validation. 

We supply `cross_val_score` with 3 mandatory parameters:

* The machine learning model
* The `X_train`
* `X_test`
  
Additionally you can use `cv` to specify how many folds and you can pick a `score` parameter, which is the result that will be reported to you as the performance on each fold.

In [ ]:
from sklearn.model_selection import cross_val_score

lin_reg_cv_results = cross_val_score(lin_reg_pipe, X_train, y_train, cv=5, scoring= "neg_root_mean_squared_error")
lin_reg_cv_results

In [ ]:
np.mean(lin_reg_cv_results)

In [ ]:
decision_tree_cv_results = cross_val_score(decision_tree_pipe, X_train, y_train, cv=5, scoring= "neg_root_mean_squared_error")
decision_tree_cv_results

In [ ]:
np.mean(decision_tree_cv_results)

##### ❓ Interpret these values. Which model performs better?

##### ❓ Contrast it to the performance seen in the beginning of this notebook of these two models. Think about overfitting, underfitting and so on.

##### ❓ Your turn: experiment with different models and setups. Try out the new pre processing pipelines with a variety of machine learning models and cross validation.

In [ ]:
# Use these together with the previous models. Make new pipelines for each of them.

country_map # Add this to the front of your pipeline
preprocessing_poly
preprocessing_bins
preprocessing_poly_bins
